In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
datasets=['dataset.csv',
'symptom_Description.csv',
'symptom_precaution.csv',
'Symptom-severity.csv']
path='./dataset/'
df_symptoms = pd.read_csv(path+datasets[0])
df_diseases = pd.read_csv(path+datasets[1])
df_precautions = pd.read_csv(path+datasets[2])
df_symps_weights = pd.read_csv(path+datasets[3]) # Weights for symptoms, or labelEncoder, NOt that Relevant for NLP... so far



In [3]:
print(df_symptoms.shape)
df_symptoms=df_symptoms.fillna("")
for col in df_symptoms.drop('Disease', axis=1).columns:
    df_symptoms[col]=df_symptoms[col].str.replace("_"," ")
df_symptoms.head()

(4920, 18)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin rash,nodal skin eruptions,dischromic patches,,,,,,,,,,,,,
1,Fungal infection,skin rash,nodal skin eruptions,dischromic patches,,,,,,,,,,,,,,
2,Fungal infection,itching,nodal skin eruptions,dischromic patches,,,,,,,,,,,,,,
3,Fungal infection,itching,skin rash,dischromic patches,,,,,,,,,,,,,,
4,Fungal infection,itching,skin rash,nodal skin eruptions,,,,,,,,,,,,,,


REviewing this: Symptoms, contain underscores as spaces. We should replace them

In [4]:
df_diseases.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [5]:
df_precautions.head()

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [6]:
df_symps_weights.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [11]:
## Create a relation between the 3 main datasets, which is Disease and symptom
df_test = pd.DataFrame()
df_test = pd.merge(df_symptoms,df_diseases, on='Disease')
df_test = pd.merge(df_test,df_precautions, on='Disease')


df_test.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,...,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17,Description,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Fungal infection,itching,skin rash,nodal skin eruptions,dischromic patches,,,,,,...,,,,,,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
1,Fungal infection,skin rash,nodal skin eruptions,dischromic patches,,,,,,,...,,,,,,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
2,Fungal infection,itching,nodal skin eruptions,dischromic patches,,,,,,,...,,,,,,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
3,Fungal infection,itching,skin rash,dischromic patches,,,,,,,...,,,,,,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
4,Fungal infection,itching,skin rash,nodal skin eruptions,,,,,,,...,,,,,,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths


In [9]:

# Group by 'Disease' and aggregate the symptoms using a custom function

# SO far I can use the dataframe as is, and later use it on an aggregated one
symptoms_set=set()

def aggregate_symptoms(symptoms):
    symptoms = [symptom for symptom in symptoms if symptom != '']
    [symptoms_set.add(x.strip()) for x in symptoms]
    return '|'.join(symptoms)

aggregated_df = df_symptoms.groupby('Disease').agg(aggregate_symptoms).reset_index()

# Rename columns to remove the 'Symptom_' prefix
aggregated_df.columns = ['Disease'] + [f'Symptom_{i}' for i in range(1, aggregated_df.shape[1])]

aggregated_df


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,(vertigo) Paroymsal Positional Vertigo,vomiting| vomiting| headache| vomiting| vomit...,headache| headache| nausea| nausea| headache|...,nausea| nausea| spinning movements| spinning ...,spinning movements| spinning movements| loss ...,loss of balance| loss of balance| unsteadines...,unsteadiness| unsteadiness| unsteadiness| uns...,,,,,,,,,,,
1,AIDS,muscle wasting| patches in throat| muscle was...,patches in throat| high fever| high fever| pa...,high fever| extra marital contacts| extra mar...,extra marital contacts| extra marital contact...,,,,,,,,,,,,,
2,Acne,skin rash| skin rash| pus filled pimples| ski...,pus filled pimples| pus filled pimples| black...,blackheads| blackheads| scurring| scurring| s...,scurring| scurring| scurring| scurring| scurr...,,,,,,,,,,,,,
3,Alcoholic hepatitis,vomiting| vomiting| yellowish skin| vomiting|...,yellowish skin| yellowish skin| abdominal pai...,abdominal pain| abdominal pain| swelling of s...,swelling of stomach| swelling of stomach| dis...,distention of abdomen| distention of abdomen|...,history of alcohol consumption| history of al...,fluid overload| fluid overload| fluid overloa...,,,,,,,,,,
4,Allergy,continuous sneezing| shivering| continuous sn...,shivering| chills| chills| shivering| shiveri...,chills| watering from eyes| watering from eye...,watering from eyes| watering from eyes| water...,,,,,,,,,,,,,
5,Arthritis,muscle weakness| muscle weakness| stiff neck|...,stiff neck| stiff neck| swelling joints| swel...,swelling joints| swelling joints| movement st...,movement stiffness| movement stiffness| painf...,painful walking| painful walking| painful wal...,,,,,,,,,,,,
6,Bronchial Asthma,fatigue| cough| fatigue| fatigue| fatigue| fa...,cough| high fever| high fever| cough| cough| ...,high fever| breathlessness| breathlessness| b...,breathlessness| family history| family histor...,family history| mucoid sputum| mucoid sputum|...,mucoid sputum| mucoid sputum| mucoid sputum| ...,,,,,,,,,,,
7,Cervical spondylosis,back pain| back pain| weakness in limbs| back...,weakness in limbs| weakness in limbs| neck pa...,neck pain| neck pain| dizziness| dizziness| d...,dizziness| dizziness| loss of balance| loss o...,loss of balance| loss of balance| loss of bal...,,,,,,,,,,,,
8,Chicken pox,itching| skin rash|itching|itching|itching|itc...,skin rash| fatigue| fatigue| skin rash| skin ...,fatigue| lethargy| lethargy| lethargy| fatigu...,lethargy| high fever| high fever| high fever|...,high fever| headache| headache| headache| hea...,headache| loss of appetite| loss of appetite|...,loss of appetite| mild fever| mild fever| mil...,mild fever| swelled lymph nodes| swelled lymp...,swelled lymph nodes| malaise| malaise| malais...,malaise| red spots over body| red spots over ...,red spots over body| red spots over body| red...,,,,,,
9,Chronic cholestasis,itching| vomiting|itching|itching|itching|itch...,vomiting| yellowish skin| yellowish skin| vom...,yellowish skin| nausea| nausea| nausea| yello...,nausea| loss of appetite| loss of appetite| l...,loss of appetite| abdominal pain| abdominal p...,abdominal pain| yellowing of eyes| yellowing ...,yellowing of eyes| yellowing of eyes| yellowi...,,,,,,,,,,


## Getting a list of all symptoms

In [13]:
print(len(symptoms_set))
symptoms_list= list(symptoms_set)

for i in range(6):
    print(symptoms_list[1*i:1*i+20])



131
['stomach pain', 'mucoid sputum', 'history of alcohol consumption', 'knee pain', 'obesity', 'blood in sputum', 'stiff neck', 'weight loss', 'cramps', 'belly pain', 'visual disturbances', 'fast heart rate', 'yellowing of eyes', 'blister', 'bloody stool', 'indigestion', 'malaise', 'polyuria', 'joint pain', 'excessive hunger']
['mucoid sputum', 'history of alcohol consumption', 'knee pain', 'obesity', 'blood in sputum', 'stiff neck', 'weight loss', 'cramps', 'belly pain', 'visual disturbances', 'fast heart rate', 'yellowing of eyes', 'blister', 'bloody stool', 'indigestion', 'malaise', 'polyuria', 'joint pain', 'excessive hunger', 'increased appetite']
['history of alcohol consumption', 'knee pain', 'obesity', 'blood in sputum', 'stiff neck', 'weight loss', 'cramps', 'belly pain', 'visual disturbances', 'fast heart rate', 'yellowing of eyes', 'blister', 'bloody stool', 'indigestion', 'malaise', 'polyuria', 'joint pain', 'excessive hunger', 'increased appetite', 'internal itching']
['k

In [89]:
symptoms_set

{'abdominal pain',
 'abnormal menstruation',
 'acidity',
 'acute liver failure',
 'altered sensorium',
 'anxiety',
 'back pain',
 'belly pain',
 'blackheads',
 'bladder discomfort',
 'blister',
 'blood in sputum',
 'bloody stool',
 'blurred and distorted vision',
 'breathlessness',
 'brittle nails',
 'bruising',
 'burning micturition',
 'chest pain',
 'chills',
 'cold hands and feets',
 'coma',
 'congestion',
 'constipation',
 'continuous feel of urine',
 'continuous sneezing',
 'cough',
 'cramps',
 'dark urine',
 'dehydration',
 'depression',
 'diarrhoea',
 'dischromic  patches',
 'distention of abdomen',
 'dizziness',
 'drying and tingling lips',
 'enlarged thyroid',
 'excessive hunger',
 'extra marital contacts',
 'family history',
 'fast heart rate',
 'fatigue',
 'fluid overload',
 'foul smell of urine',
 'headache',
 'high fever',
 'hip joint pain',
 'history of alcohol consumption',
 'increased appetite',
 'indigestion',
 'inflammatory nails',
 'internal itching',
 'irregular sug

#### Testing NLPm

In [65]:
txt_user_data = " My name is Ivan, male, I have 18 years, and I am 1.65 meters"
txt_my_symptoms = "I have a headache, maybe a fever. Also some joint pain, and itchyness. I went to The jungle for 10 days, I eat raw food, and some insects bit me, my skin looks yellowish." #Dengue
txt_my_symptoms = "I'm not feeling well, a bit sick. My throat is itchy, my nose is runny. I don't have energy." #cold


In [ ]:
import spacy
from nltk.corpus import stopwords
stopword = stopwords.words('english')
nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load("en_core_web_lg")

In [63]:
docs= nlp(txt_my_symptoms)
print(docs)
for d  in docs.noun_chunks:
    print(d)

I'm not feeling well, a bit sick. My throat is itchy, my nose is runny. I don't have energy.
I
My throat
my nose
I
energy


In [62]:
docs_me = nlp(txt_user_data)
for d in docs_me.ents:
    print(d)

Ivan
18 years
1.65 meters


In [64]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def clean_text(text):
    # Remove special characters, extra spaces, and formatting
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens


def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens


def pos_tagging(tokens):
    tagged_tokens = nltk.pos_tag(tokens)
    return tagged_tokens


def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities


def dependency_parsing(text):
    doc = nlp(text)
    dependencies = [(token.text, token.dep_, token.head.text) for token in doc]
    return dependencies



def extract_keywords(texts):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Use cosine_similarities matrix to find important keywords
    # For example, words with high TF-IDF scores or TextRank scores


In [73]:
raw_text=txt_my_symptoms
cleaned_text = clean_text(raw_text)
print(cleaned_text)
# Tokenization: Split the cleaned text into individual words or tokens.
tokens = tokenize_text(cleaned_text)
print("\n tokens:",tokens)

# Stopword Removal: Remove common stopwords that don't provide meaningful information.
filtered_tokens = remove_stopwords(tokens)
print("\n filtered tokens:",filtered_tokens)
# Part-of-Speech Tagging: Identify the parts of speech of each token.
tagged_tokens = pos_tagging(filtered_tokens)
print("\n tagged tokens:",tagged_tokens)
# Named Entity Recognition (NER): Extract medical entities like symptoms and descriptions.
entities = extract_entities(cleaned_text)
print("\n >entities:",entities)
# Dependency Parsing (Optional): Analyze grammatical relationships between words.
dependencies = dependency_parsing(cleaned_text)
print("\n >dependencies:",dependencies)
# Keyword Extraction (Optional): Identify important keywords using TF-IDF or TextRank.
keywords = extract_keywords(tokens)


print("\n >keywords:",keywords)

Im not feeling well a bit sick My throat is itchy my nose is runny I dont have energy

 tokens: ['Im', 'not', 'feeling', 'well', 'a', 'bit', 'sick', 'My', 'throat', 'is', 'itchy', 'my', 'nose', 'is', 'runny', 'I', 'dont', 'have', 'energy']

 filtered tokens: ['Im', 'feeling', 'well', 'bit', 'sick', 'throat', 'itchy', 'nose', 'runny', 'dont', 'energy']

 tagged tokens: [('Im', 'NNP'), ('feeling', 'VBG'), ('well', 'RB'), ('bit', 'RB'), ('sick', 'JJ'), ('throat', 'NN'), ('itchy', 'NN'), ('nose', 'JJ'), ('runny', 'JJ'), ('dont', 'NN'), ('energy', 'NN')]

 >entities: []

 dependencies: [('I', 'nsubj', 'feeling'), ('m', 'aux', 'feeling'), ('not', 'neg', 'feeling'), ('feeling', 'ROOT', 'feeling'), ('well', 'advmod', 'feeling'), ('a', 'det', 'bit'), ('bit', 'npadvmod', 'sick'), ('sick', 'acomp', 'feeling'), ('My', 'poss', 'throat'), ('throat', 'nsubj', 'is'), ('is', 'ccomp', 'feeling'), ('itchy', 'acomp', 'is'), ('my', 'poss', 'nose'), ('nose', 'nsubj', 'is'), ('is', 'ccomp', 'is'), ('runny', 

In [76]:
# PErsonal info
raw_text=txt_user_data
print("raw_text:",raw_text)

ents=extract_entities(raw_text)
print("Entities",ents)

raw_text:  My name is Ivan, male, I have 18 years, and I am 1.65 meters
Entities [('Ivan', 'PERSON'), ('18 years', 'DATE'), ('1.65 meters', 'QUANTITY')]


# Train nlp with symptoms


In [140]:
SYMPTOMS_TRAIN_DATA = [
    ("There's a red sore around my nose that's been bothering me.", {
        'entities': [(10, 28, 'SYMPTOM')]
    }),
    ("Even in cold weather, I find myself sweating excessively.", {
        'entities': [(34, 42, 'SYMPTOM')]
    }),
    ("I've noticed some scurring on my skin recently.", {
        'entities': [(18, 26, 'SYMPTOM')]
    }),
    ("I have this continuous feel of urine, even after using the bathroom.", {
        'entities': [(13, 35, 'SYMPTOM')]
    }),
    ("I keep coughing up phlegm every morning.", {
        'entities': [(18, 24, 'SYMPTOM')]
    }),
    ("I experience a lot of pain during bowel movements.", {
        'entities': [(24, 49, 'SYMPTOM')]
    }),
    ("At night, I often get palpitations that make it hard to sleep.", {
        'entities': [(19, 31, 'SYMPTOM')]
    }),
    ("I've been going to the bathroom more frequently; I think it might be polyuria.", {
        'entities': [(65, 73, 'SYMPTOM')]
    }),
    ("My lips have been drying and tingling a lot lately.", {
        'entities': [(14, 35, 'SYMPTOM')]
    }),
    ("I feel a burning sensation during micturition.", {
        'entities': [(8, 27, 'SYMPTOM')]
    }),
    ("My nails look different, almost like they're inflammatory.", {
        'entities': [(45, 62, 'SYMPTOM')]
    }),
    ("Lately, I've been feeling muscle weakness, especially in my arms.", {
        'entities': [(24, 38, 'SYMPTOM')]
    }),
    ("Every time I eat something, I feel nausea.", {
        'entities': [(32, 38, 'SYMPTOM')]
    }),
    ("The doctor said my thyroid is enlarged.", {
        'entities': [(22, 36, 'SYMPTOM')]
    }),
    ("My child has a high fever since yesterday.", {
        'entities': [(16, 26, 'SYMPTOM')]
    }),
    ("After eating street food, I had severe diarrhoea.", {
        'entities': [(36, 45, 'SYMPTOM')]
    }),
    ("My hands and feets are always cold, no matter the weather.", {
        'entities': [(3, 23, 'SYMPTOM')]
    }),
    ("He looks unwell, almost a toxic look like typhos.", {
        'entities': [(24, 42, 'SYMPTOM')]
    }),
    ("After the accident, I've felt movement stiffness in my legs.", {
        'entities': [(30, 47, 'SYMPTOM')]
    }),
    ("My face has several pus filled pimples that are painful.", {
        'entities': [(19, 36, 'SYMPTOM')]
    }),

    ("Even in air-conditioned rooms, I keep sweating.", {
        'entities': [(39, 47, 'SYMPTOM')]
    }),
    ("There's this unusual scurring on my arm.", {
        'entities': [(22, 30, 'SYMPTOM')]
    }),
    ("I constantly have this continuous feel of urine.", {
        'entities': [(22, 44, 'SYMPTOM')]
    }),
    ("Every morning, there's phlegm in my throat.", {
        'entities': [(22, 28, 'SYMPTOM')]
    }),
    ("I've been experiencing pain during bowel movements lately.", {
        'entities': [(23, 48, 'SYMPTOM')]
    }),
    ("Sometimes, out of nowhere, I get these palpitations.", {
        'entities': [(37, 49, 'SYMPTOM')]
    }),
    ("I suspect I might be suffering from polyuria.", {
        'entities': [(35, 43, 'SYMPTOM')]
    }),
    ("I've noticed my lips drying and tingling quite frequently.", {
        'entities': [(20, 41, 'SYMPTOM')]
    }),
    ("There's a burning sensation every time I'm in micturition.", {
        'entities': [(9, 28, 'SYMPTOM')]
    }),
    ("Recently, my nails seem to be quite inflammatory.", {
        'entities': [(30, 47, 'SYMPTOM')]
    }),
    ("My legs give out due to this muscle weakness.", {
        'entities': [(29, 43, 'SYMPTOM')]
    }),
    ("I can't eat much without feeling nausea.", {
        'entities': [(29, 35, 'SYMPTOM')]
    }),
    ("The ultrasound showed an enlarged thyroid.", {
        'entities': [(25, 39, 'SYMPTOM')]
    }),
    ("He's been unwell with a high fever for three days.", {
        'entities': [(23, 33, 'SYMPTOM')]
    }),
    ("She's been in bed with severe diarrhoea.", {
        'entities': [(28, 37, 'SYMPTOM')]
    }),
    ("Despite wearing gloves, my hands and feets are cold.", {
        'entities': [(29, 49, 'SYMPTOM')]
    }),
    ("His appearance has that toxic look of typhos.", {
        'entities': [(23, 41, 'SYMPTOM')]
    }),
    ("Since the fall, I've experienced movement stiffness.", {
        'entities': [(31, 48, 'SYMPTOM')]
    }),
    ("I've been breaking out with pus filled pimples on my chin.", {
        'entities': [(26, 43, 'SYMPTOM')]
    }),
    ("I keep coughing and sometimes I see mucoid sputum.", {
        'entities': [(39, 52, 'SYMPTOM')]
    }),

    ("I noticed some strange scurring on my back.", {
        'entities': [(24, 32, 'SYMPTOM')]
    }),
    ("Even after going to the bathroom, I still have this continuous feel of urine.", {
        'entities': [(53, 75, 'SYMPTOM')]
    }),
    ("It's hard to swallow because there's so much phlegm.", {
        'entities': [(42, 48, 'SYMPTOM')]
    }),
    ("The pain during bowel movements is becoming unbearable.", {
        'entities': [(4, 29, 'SYMPTOM')]
    }),
    ("My heart races suddenly, these palpitations are concerning.", {
        'entities': [(30, 42, 'SYMPTOM')]
    }),
    ("I've been urinating frequently, I fear it's polyuria.", {
        'entities': [(42, 50, 'SYMPTOM')]
    }),
    ("My lips are constantly drying and tingling, it's very uncomfortable.", {
        'entities': [(21, 42, 'SYMPTOM')]
    }),
    ("Everytime I try to urinate, I feel burning micturition.", {
        'entities': [(33, 52, 'SYMPTOM')]
    }),
    ("Are these inflammatory nails a sign of some disease?", {
        'entities': [(9, 26, 'SYMPTOM')]
    }),
    ("Climbing stairs is tough due to my muscle weakness.", {
        'entities': [(31, 45, 'SYMPTOM')]
    }),
    ("I had one slice of cake and immediately felt nausea.", {
        'entities': [(44, 50, 'SYMPTOM')]
    }),
    ("My doctor pointed out that I have an enlarged thyroid.", {
        'entities': [(37, 51, 'SYMPTOM')]
    }),
    ("I'm worried about her, she has a high fever that's not going down.", {
        'entities': [(32, 42, 'SYMPTOM')]
    }),
    ("Stomach issues are bad, but this diarrhoea is the worst.", {
        'entities': [(33, 42, 'SYMPTOM')]
    }),
    ("No matter how many socks I wear, my cold hands and feets won't warm up.", {
        'entities': [(43, 63, 'SYMPTOM')]
    }),
    ("She had a distinct toxic look, almost like typhos.", {
        'entities': [(16, 34, 'SYMPTOM')]
    }),
    ("It's not just pain, it's movement stiffness in my arm.", {
        'entities': [(25, 42, 'SYMPTOM')]
    }),
    ("My forehead is breaking out with pus filled pimples.", {
        'entities': [(32, 49, 'SYMPTOM')]
    }),
    ("In the mornings, I often cough up mucoid sputum.", {
        'entities': [(37, 50, 'SYMPTOM')]
    }),
    ("I've been having painful ulcers on my tongue recently.", {
        'entities': [(25, 39, 'SYMPTOM')]
    }),
    
    
    ("I keep having the continuous feel of urine, even after using the bathroom.", {
        'entities': [(16, 38, 'SYMPTOM')]
    }),
    ("I can't clear my throat; there's too much phlegm.", {
        'entities': [(39, 45, 'SYMPTOM')]
    }),
    ("Recently, the pain during bowel movements has intensified.", {
        'entities': [(13, 38, 'SYMPTOM')]
    }),
    ("Out of the blue, I'll start having palpitations and it's scary.", {
        'entities': [(32, 44, 'SYMPTOM')]
    }),
    ("I've been visiting the washroom more than usual, maybe it's polyuria.", {
        'entities': [(57, 65, 'SYMPTOM')]
    }),
    ("It's strange how my lips keep drying and tingling all the time.", {
        'entities': [(29, 50, 'SYMPTOM')]
    }),
    ("I experience burning micturition whenever I try to urinate.", {
        'entities': [(12, 31, 'SYMPTOM')]
    }),
    ("The skin around my nails is red and looks like inflammatory nails.", {
        'entities': [(45, 62, 'SYMPTOM')]
    }),
    ("It's harder to lift things now because of the muscle weakness.", {
        'entities': [(46, 60, 'SYMPTOM')]
    }),
    ("Whenever I eat oily food, I get severe nausea.", {
        'entities': [(37, 43, 'SYMPTOM')]
    }),
    ("My recent scans showed an enlarged thyroid.", {
        'entities': [(23, 37, 'SYMPTOM')]
    }),
    ("She's been down with a really high fever for days.", {
        'entities': [(28, 38, 'SYMPTOM')]
    }),
    ("He can't leave the house because of his severe diarrhoea.", {
        'entities': [(46, 55, 'SYMPTOM')]
    }),
    ("I've been wearing gloves but still have cold hands and feets.", {
        'entities': [(44, 64, 'SYMPTOM')]
    }),
    ("The patient has a toxic look, it's likely typhos.", {
        'entities': [(16, 34, 'SYMPTOM')]
    }),
    ("I feel movement stiffness, especially in the mornings.", {
        'entities': [(7, 24, 'SYMPTOM')]
    }),
    ("I hate these pus filled pimples that have appeared on my face.", {
        'entities': [(10, 27, 'SYMPTOM')]
    }),
    ("When I cough, I sometimes produce mucoid sputum.", {
        'entities': [(35, 48, 'SYMPTOM')]
    }),
    ("There are these painful ulcers on my tongue that won't heal.", {
        'entities': [(21, 35, 'SYMPTOM')]
    }),
    ("Lately, I've been avoiding stairs because of the painful walking.", {
        'entities': [(50, 64, 'SYMPTOM')]
    }),

    ("The constant phlegm in my throat is bothering me.", {
        'entities': [(13, 19, 'SYMPTOM')]
    }),
    ("Every morning, I experience pain during bowel movements.", {
        'entities': [(25, 50, 'SYMPTOM')]
    }),
    ("At random moments, I'll feel these palpitations.", {
        'entities': [(32, 44, 'SYMPTOM')]
    }),
    ("Increased frequency of urination might be polyuria.", {
        'entities': [(42, 50, 'SYMPTOM')]
    }),
    ("Recently, my lips have been drying and tingling a lot.", {
        'entities': [(25, 46, 'SYMPTOM')]
    }),
    ("Every time I go to the washroom, I feel burning micturition.", {
        'entities': [(42, 61, 'SYMPTOM')]
    }),
    ("I noticed some redness and inflammation, could it be inflammatory nails?", {
        'entities': [(54, 71, 'SYMPTOM')]
    }),
    ("Lifting things has become difficult due to muscle weakness.", {
        'entities': [(41, 55, 'SYMPTOM')]
    }),
    ("The smell of the food made me feel sudden nausea.", {
        'entities': [(36, 42, 'SYMPTOM')]
    }),
    ("The doctor mentioned that I might have an enlarged thyroid.", {
        'entities': [(42, 56, 'SYMPTOM')]
    }),
    ("His high fever hasn't subsided for two days.", {
        'entities': [(4, 14, 'SYMPTOM')]
    }),
    ("The food poisoning led to severe diarrhoea.", {
        'entities': [(31, 40, 'SYMPTOM')]
    }),
    ("Even indoors, my cold hands and feets don't seem to get warm.", {
        'entities': [(14, 34, 'SYMPTOM')]
    }),
    ("She had a pale, toxic look on her face.", {
        'entities': [(16, 26, 'SYMPTOM')]
    }),
    ("The symptoms are pointing towards typhos.", {
        'entities': [(33, 39, 'SYMPTOM')]
    }),
    ("After the injury, I've felt movement stiffness in my leg.", {
        'entities': [(26, 43, 'SYMPTOM')]
    }),
    ("This breakout on my face is full of pus filled pimples.", {
        'entities': [(37, 54, 'SYMPTOM')]
    }),
    ("The cough is productive, often bringing up mucoid sputum.", {
        'entities': [(46, 59, 'SYMPTOM')]
    }),
    ("These spicy foods seem to cause ulcers on my tongue.", {
        'entities': [(33, 47, 'SYMPTOM')]
    }),
    ("The accident left me with painful walking for weeks.", {
        'entities': [(24, 38, 'SYMPTOM')]
    }),
    ("The illness caused severe muscle wasting in him.", {
        'entities': [(26, 40, 'SYMPTOM')]
    }),

    ("I've been experiencing pain during bowel movements recently.", {
        'entities': [(21, 46, 'SYMPTOM')]
    }),
    ("Sometimes, out of nowhere, I'll get these palpitations.", {
        'entities': [(37, 49, 'SYMPTOM')]
    }),
    ("The doctor suggested I might have polyuria due to increased water intake.", {
        'entities': [(30, 38, 'SYMPTOM')]
    }),
    ("My lips are constantly drying and tingling, especially in this weather.", {
        'entities': [(21, 42, 'SYMPTOM')]
    }),
    ("It hurts when I urinate; I think it's burning micturition.", {
        'entities': [(34, 53, 'SYMPTOM')]
    }),
    ("Could the redness around my nails be inflammatory nails?", {
        'entities': [(37, 54, 'SYMPTOM')]
    }),
    ("Recently, even lifting light objects shows my muscle weakness.", {
        'entities': [(43, 57, 'SYMPTOM')]
    }),
    ("Every time I think about it, I get waves of nausea.", {
        'entities': [(43, 49, 'SYMPTOM')]
    }),
    ("The tests confirmed that my gland was an enlarged thyroid.", {
        'entities': [(38, 52, 'SYMPTOM')]
    }),
    ("She's been bedridden with a high fever for the past three days.", {
        'entities': [(25, 35, 'SYMPTOM')]
    }),
    ("After that meal, I've had terrible diarrhoea.", {
        'entities': [(28, 37, 'SYMPTOM')]
    }),
    ("No matter how many blankets I use, my cold hands and feets remain icy.", {
        'entities': [(41, 61, 'SYMPTOM')]
    }),
    ("She was very ill, presenting a toxic look suggestive of typhos.", {
        'entities': [(28, 38, 'SYMPTOM'), (54, 60, 'SYMPTOM')]
    }),
    ("I've noticed movement stiffness in my knee after the fall.", {
        'entities': [(12, 29, 'SYMPTOM')]
    }),
    ("There are several pus filled pimples on my back.", {
        'entities': [(16, 33, 'SYMPTOM')]
    }),
    ("Whenever I cough, there's a lot of mucoid sputum.", {
        'entities': [(34, 47, 'SYMPTOM')]
    }),
    ("Spicy foods seem to result in ulcers on my tongue.", {
        'entities': [(30, 44, 'SYMPTOM')]
    }),
    ("Ever since the accident, I've had a painful walking experience.", {
        'entities': [(32, 46, 'SYMPTOM')]
    }),
    ("The disease led to noticeable muscle wasting over a few months.", {
        'entities': [(28, 42, 'SYMPTOM')]
    }),
    ("Lately, there's been constant watering from my eyes.", {
        'entities': [(26, 44, 'SYMPTOM')]
    }),

    # Negative examples
#     ("I went to the market yesterday.", {
#         'entities': []
#     }),
#     ("The movie last night was so thrilling!", {
#         'entities': []
#     }),
#     ("Have you read the latest book by Rowling?", {
#         'entities': []
#     }),
#     ("It's important to drink water daily.", {
#         'entities': []
#     }),
#     ("The weather is quite unpredictable these days.", {
#         'entities': []
#     }),
#     ("I think I forgot my phone at the cafe.", {
#         'entities': []
#     }),
#     ("Can you recommend a good restaurant nearby?", {
#         'entities': []
#     }),
#     ("My favorite color is blue.", {
#         'entities': []
#     }),
#     ("Traveling to Europe has always been my dream.", {
#         'entities': []
#     }),
#     ("The soccer game was quite intense.", {
#         'entities': []
#     }),
#     ("I went to the market yesterday.", {
#         'entities': []
#     }),
#     ("The movie last night was so thrilling!", {
#         'entities': []
#     }),
#     ("Have you read the latest book by Rowling?", {
#         'entities': []
#     }),
#     ("It's important to drink water daily.", {
#         'entities': []
#     }),
#     ("The weather is quite unpredictable these days.", {
#         'entities': []
#     }),
#     ("I think I forgot my phone at the cafe.", {
#         'entities': []
#     }),
#     ("Can you recommend a good restaurant nearby?", {
#         'entities': []
#     }),
#     ("My favorite color is blue.", {
#         'entities': []
#     }),
#     ("Traveling to Europe has always been my dream.", {
#         'entities': []
#     }),
#     ("The soccer game was quite intense.", {
#         'entities': []
#     }),
#     ("How's the stock market performing today?", {
#         'entities': []
#     }),
#     ("I need to recharge my phone soon.", {
#         'entities': []
#     }),
#     ("The new software update is really useful.", {
#         'entities': []
#     }),
#     ("She's planning a trip to the mountains next month.", {
#         'entities': []
#     }),
#     ("How many languages can you speak?", {
#         'entities': []
#     }),
#     ("I've been learning to play the guitar.", {
#         'entities': []
#     }),
#     ("This cake tastes amazing!", {
#         'entities': []
#     }),
#     ("The concert tickets sold out within minutes.", {
#         'entities': []
#     }),
#     ("I love spending time by the beach.", {
#         'entities': []
#     }),
#     ("The art exhibition displayed some remarkable pieces.", {
#         'entities': []
#     }),
#     ("I took a long walk in the park.", {
#         'entities': []
#     }),
#     ("She baked a delicious apple pie for dinner.", {
#         'entities': []
#     }),
#     ("It's going to rain tomorrow according to the forecast.", {
#         'entities': []
#     }),
#     ("I've always wanted to visit the Eiffel Tower.", {
#         'entities': []
#     }),
#     ("They went on a road trip across the country last summer.", {
#         'entities': []
#     }),
#     ("He's an expert in ancient Greek philosophy.", {
#         'entities': []
#     }),
#     ("Can you believe it's already December?", {
#         'entities': []
#     }),
#     ("The bookshop on the corner has a sale this weekend.", {
#         'entities': []
#     }),
#     ("She's taking piano lessons from a renowned musician.", {
#         'entities': []
#     }),
#     ("The new video game release caused a lot of excitement.", {
#         'entities': []
#     }),
#     ("I've been trying out different coffee beans to find my favorite.", {
#         'entities': []
#     }),
#     ("This year's music festival lineup is incredible.", {
#         'entities': []
#     }),
#     ("He's a big fan of classic rock bands from the 70s.", {
#         'entities': []
#     }),
#     ("The documentary on ocean life was truly eye-opening.", {
#         'entities': []
#     }),
#     ("She's learning French for her upcoming trip to Paris.", {
#         'entities': []
#     }),
#     ("I never miss watching the annual meteor shower.", {
#         'entities': []
#     }),
#     ("His collection of vintage cars is quite impressive.", {
#         'entities': []
#     }),
#     ("I've always been fascinated by the mysteries of outer space.", {
#         'entities': []
#     }),
#     ("She loves dancing to Latin music.", {
#         'entities': []
#     }),
#     ("The museum's exhibit on Egyptian mummies is a must-see.", {
#         'entities': []
#     }),
]


#SYMPTOMS_TRAIN_DATA


In [141]:
import spacy
from spacy.training.example import Example
import random

# Load an existing pre-trained model
nlp = spacy.load("en_core_web_lg")

# Get the NER component
ner = nlp.get_pipe("ner")

# Add the new label to the NER component
ner.add_label("SYMPTOM")

# Train the NER model
optimizer = nlp.resume_training()
for i in range(10):
    random.shuffle(SYMPTOMS_TRAIN_DATA)
    losses = {}
    for text, annotations in SYMPTOMS_TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)
    print("Losses:", losses)


Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 20.80483166862083}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 18.20975922792859}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 15.86622006766806}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 21.49978158171939}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 9.395415113374684}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 20.764005797054757}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 11.42242734200415}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 10.137737554735706}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 8.596983418744422}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 9.37812443042784}


In [142]:

SYMP_MODEL = "models/symp_model"
nlp.to_disk(SYMP_MODEL)

In [145]:
# loaded_nlp = spacy.load(SYMP_MODEL)
text = "The patient experienced severe pain in the chest."
text = "I have a headache, maybe a fever. Also some joint pain, and itchyness. I went to The jungle for 10 days, I eat raw food, and some insects bit me, my skin looks yellowish." #Dengue

doc = loaded_nlp(text)
print(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


I have a headache, maybe a fever. Also some joint pain, and itchyness. I went to The jungle for 10 days, I eat raw food, and some insects bit me, my skin looks yellowish.
itchyness 60 69 SYMPTOM


# How to create a dataset

In [139]:
def generate_training_data(text, symptoms_list):
    entities = []
    for symptom in symptoms_list:
        start_index = text.find(symptom)
        if start_index != -1:
            end_index = start_index + len(symptom)
            entities.append((start_index, end_index, 'SYMPTOM'))
    return (text, {"entities": entities})

text = "I have had a severe headache and cough for three days now."
training_example = generate_training_data(text, symptoms_list)
print(training_example)

('I have had a severe headache and cough for three days now.', {'entities': [(20, 28, 'SYMPTOM'), (33, 38, 'SYMPTOM')]})


In [148]:
symptoms_list

['red sore around nose',
 'sweating',
 'scurring',
 'continuous feel of urine',
 'phlegm',
 'pain during bowel movements',
 'palpitations',
 'polyuria',
 'drying and tingling lips',
 'burning micturition',
 'inflammatory nails',
 'muscle weakness',
 'nausea',
 'enlarged thyroid',
 'high fever',
 'diarrhoea',
 'cold hands and feets',
 'toxic look (typhos)',
 'movement stiffness',
 'pus filled pimples',
 'mucoid sputum',
 'ulcers on tongue',
 'painful walking',
 'muscle wasting',
 'watering from eyes',
 'skin peeling',
 'lack of concentration',
 'small dents in nails',
 'internal itching',
 'swelling of stomach',
 'redness of eyes',
 'coma',
 'swollen blood vessels',
 'unsteadiness',
 'puffy face and eyes',
 'prominent veins on calf',
 'weakness in limbs',
 'blood in sputum',
 'weight loss',
 'headache',
 'loss of appetite',
 'obesity',
 'yellowish skin',
 'swelled lymph nodes',
 'mild fever',
 'brittle nails',
 'skin rash',
 'history of alcohol consumption',
 'fluid overload',
 'anxiety

In [68]:
sentences_to_train=[
"She noticed a red sore around nose and wondered if it was from her new face cream.",
"He was sweating profusely even though the room was cool.",
"The mice were scurrying across the kitchen floor late at night.",
"She had a continuous feel of urine, making her uncomfortable during the meeting.",
"He woke up with a throat full of phlegm every morning.",
"Pain during bowel movements can be a sign of various gastrointestinal issues.",
"He felt palpitations whenever he drank too much coffee.",
"Polyuria can be a symptom of diabetes, where one urinates more than usual.",
"She constantly licked her drying and tingling lips, pondering the cause.",
"Burning micturition made him dread every visit to the bathroom.",
"She observed inflammatory nails and decided to visit a dermatologist.",
"After the long hike, he experienced significant muscle weakness.",
"The overwhelming nausea kept her bedridden for most of the day.",
"A noticeable enlarged thyroid prompted a visit to the endocrinologist.",
"The child had a high fever, which left the parents very concerned.",
"Travelling often brought about bouts of diarrhoea for him.",
"She wore gloves as her cold hands and feet always made her feel frigid.",
"He had a toxic look (typhos) that worried all who saw him.",
"Movement stiffness in the morning is a common symptom in arthritis.",
"Pus-filled pimples on her face made her reconsider her skincare routine.",
"After the intense workout, he was sweating more than he ever had before.",
"The sound of tiny feet scurring across the attic floor kept her awake at night.",
"The continuous feel of urine made her trips to the bathroom frequent and urgent.",
"Every winter morning, he had to clear his throat of thick phlegm.",
"He made an appointment with his gastroenterologist due to the recurring pain during bowel movements.",
"Loud noises always induced palpitations in her chest, making her jump.",
"His recent bout of polyuria was concerning enough to book a doctor’s appointment.",
"The desert air caused her to constantly experience drying and tingling lips.",
"The sensation of burning micturition was unbearable, leading him to seek medical advice.",
"Her inflammatory nails were a result of a fungal infection she'd contracted.",
"Muscle weakness in his arms made lifting even light objects a challenge.",
"The bumpy boat ride led to nausea that persisted for hours.",
"An enlarged thyroid indicated a possible hormonal imbalance that needed medical attention.",
"His high fever, sustained over two days, alarmed his family.",
"After eating street food, she was plagued with severe diarrhoea.",
"Despite the warm room, she always had cold hands and feet, making her wrap them in blankets.",
"His sunken eyes and toxic look (typhos) worried everyone around him.",
"Movement stiffness in the morning prevented him from jumping out of bed like he used to.",
"The onset of pus-filled pimples right before the event was her worst nightmare.",
"The persistent cough was accompanied by mucoid sputum, signaling a potential respiratory infection.",
"The scurring of tiny insects made her hesitant to enter the old shed.",
"Due to the continuous feel of urine, he constantly searched for restrooms wherever he went.",
"Each morning, she had to deal with a buildup of phlegm that made her cough.",
"The ongoing pain during bowel movements was becoming a real concern for him.",
"Every time she got nervous about a presentation, she felt palpitations in her chest.",
"His need to address polyuria led him to frequently visit the clinic for check-ups.",
"She kept a lip balm handy, especially when her drying and tingling lips flared up in winter.",
"Experiencing burning micturition was a sensation he'd never felt before, and it was deeply unsettling.",
"Wearing damp shoes resulted in her getting inflammatory nails that ached constantly.",
"After the viral infection, he complained of muscle weakness and fatigue.",
"The smell of certain foods triggered an instant feeling of nausea in her.",
"An enlarged thyroid was evident from the bulge in her neck, prompting immediate medical tests.",
"The child's high fever was a cause for alarm, making the parents rush to the hospital.",
"She regretted eating from the roadside stall as it led to a severe bout of diarrhoea.",
"No matter how warm it was, her cold hands and feet always required extra layers.",
"His ashen face and toxic look (typhos) immediately alerted the nurse to his critical condition.",
"His movement stiffness became more pronounced in cold weather, slowing him down considerably.",
"The pus-filled pimples on her face appeared overnight, causing much distress.",
"His cough became more concerning when he started producing mucoid sputum.",
"Spicy food became a challenge for her due to the painful ulcers on her tongue.",
"Her road trip was frequently interrupted by a continuous feel of urine, making her stop at every rest area.",
"His voice sounded congested due to the phlegm that accumulated overnight.",
"A diet lacking in fiber often led him to experience pain during bowel movements.",
"Whenever she thought about the upcoming interview, she felt palpitations racing through her chest.",
"The sudden onset of polyuria raised concerns about potential underlying conditions.",
"Despite hydrating regularly, she couldn't escape the discomfort of drying and tingling lips.",
"The intense sensation of burning micturition made him wary of drinking too much water.",
"Her inflammatory nails were a painful reminder of the rainy hike where her shoes got soaked.",
"The bout of flu left him with a lingering sense of muscle weakness.",
"The motion of the boat on choppy waters induced waves of nausea in her.",
"Routine health screenings detected an enlarged thyroid, leading to further diagnostic tests.",
"The high fever he developed overnight was a clear indication of an infection.",
"After trying a new street food dish, she regretfully dealt with severe diarrhoea.",
"Even on warm summer evenings, she would curl up with blankets due to her cold hands and feet.",
"The doctor immediately recognized his toxic look as a sign of a severe infection.",
"The old injury led to movement stiffness, especially during the cold mornings.",
"On the eve of the photo shoot, she woke up to find pus-filled pimples on her chin.",
"His persistent cough was made worse by the production of mucoid sputum.",
"The ulcers on her tongue made it difficult to enjoy her favorite spicy dishes.",
"The blistered heels resulted in painful walking, forcing her to reconsider her choice of footwear.",
"Morning routines often started with him clearing out phlegm, especially during allergy season.",
"His discomfort was evident, especially with the recurrent pain during bowel movements.",
"Watching the suspenseful movie gave her palpitations, making her clutch her chest.",
"Regular trips to the restroom were becoming bothersome due to his polyuria.",
"As winter approached, she braced herself for the inevitable drying and tingling lips.",
"Drinking cranberry juice was suggested to him as a remedy for burning micturition.",
"Constant exposure to water caused her inflammatory nails, requiring a dermatologist's attention.",
"Climbing the stairs became a challenge due to his increasing muscle weakness.",
"Just the thought of riding the roller coaster filled her with overpowering nausea.",
"The visible lump on her neck turned out to be an enlarged thyroid, necessitating further medical evaluation.",
"Amidst flu season, her high fever had everyone in the household worried.",
"The aftermath of indulging in exotic foods was an unfortunate bout of diarrhoea.",
"Her cold hands and feet were an odd contrast to the warm summer day.",
"His pallor and toxic look were immediate signs to the nurse that he needed urgent care.",
"Aging brought with it movement stiffness, particularly evident when he tried to get up in the mornings.",
"The sudden outbreak of pus-filled pimples right before the event left her feeling self-conscious.",
"The appearance of mucoid sputum with his cough indicated a possible respiratory infection.",
"Talking became a painful task due to the ulcers on her tongue.",
"The sprained ankle resulted in such painful walking that she had to rely on crutches.",
"Prolonged bed rest post-surgery led to noticeable muscle wasting in his legs.",
"Absolutely, here are individual sentences for each of the terms provided:",
"His reluctance to visit the restroom was due to the consistent pain during bowel movements.",
"During stressful situations, she often felt palpitations that seemed to echo in her ears.",
"Tracking his daily fluid intake became important as he managed polyuria.",
"The mountain air was refreshing, but it caused her drying and tingling lips.",
"The burning micturition after a long day was a sign that he needed to consult a doctor.",
"After her trek in the muddy woods, she developed inflammatory nails.",
"Simple activities became arduous due to his advancing muscle weakness.",
"The winding road up the hill left her with a strong sense of nausea.",
"Regular check-ups were advised after they detected her enlarged thyroid.",
"His sudden high fever was concerning enough to warrant a visit to the hospital.",
"The spoiled seafood resulted in a distressing case of diarrhoea.",
"Despite the roaring fire, she constantly felt cold hands and feet.",
"His slumped posture and typhos alarmed his caregivers.",
"Waking up from his nap, he felt pronounced movement stiffness in his joints.",
"Just days before her wedding, she was dismayed to discover pus-filled pimples on her face.",
"Each time he coughed, the mucoid sputum hinted at a deeper respiratory issue.",
"The ulcers on her tongue made even sipping water a painful ordeal.",
"The heel injury led to such painful walking that she had to reconsider her daily jog.",
"Months of inactivity due to the lockdown showed clear signs of muscle wasting.",
"The pungent aroma of the onions led to uncontrollable watering from her eyes."

]

In [150]:
NEW_TRAIN_DATA_SENTENCES=[]
for text in sentences_to_train:
    training_example = generate_training_data(text, symptoms_list)
    print(training_example)
    NEW_TRAIN_DATA_SENTENCES.append(training_example)

('She noticed a red sore around nose and wondered if it was from her new face cream.', {'entities': [(14, 34, 'SYMPTOM')]})
('He was sweating profusely even though the room was cool.', {'entities': [(7, 15, 'SYMPTOM')]})
('The mice were scurrying across the kitchen floor late at night.', {'entities': []})
('She had a continuous feel of urine, making her uncomfortable during the meeting.', {'entities': [(10, 34, 'SYMPTOM')]})
('He woke up with a throat full of phlegm every morning.', {'entities': [(33, 39, 'SYMPTOM')]})
('Pain during bowel movements can be a sign of various gastrointestinal issues.', {'entities': []})
('He felt palpitations whenever he drank too much coffee.', {'entities': [(8, 20, 'SYMPTOM')]})
('Polyuria can be a symptom of diabetes, where one urinates more than usual.', {'entities': []})
('She constantly licked her drying and tingling lips, pondering the cause.', {'entities': [(26, 50, 'SYMPTOM')]})
('Burning micturition made him dread every visit to the bathroom.', 

#  
#  
# TEST EXTRACTING USER INFO!!!!!!

In [154]:
symptoms_list
txt_user_data = " My name is John, male, I have 18 years old, and I am 1.65 meters, weight 100 kg."
txt_my_symptoms = "I have a headache, maybe a fever. Also some joint pain, and itchyness. I went to The jungle for 10 days, I eat raw food, and some insects bit me, my skin looks yellowish." #Dengue
fultext=txt_user_data+". "+txt_my_symptoms

raw_text=fultext

ents=extract_entities(raw_text)
print("Entities",ents)

Entities [('John', 'PERSON'), ('18 years old', 'DATE'), ('1.65 meters', 'QUANTITY'), ('100 kg', 'QUANTITY'), ('10 days', 'DATE')]


In [165]:

def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities


def calculate_bmi(weight, height):
    """Calculate BMI (kg) and (m)"""
    return weight / (height ** 2)
def feet_to_meters(feet):
    return feet * 0.3048
def pounds_to_kilograms(pounds):
    return pounds * 0.453592



txt_user_data = "My name is John, male, I have 18 years old, and I am 1.65 meters, weight 100 pounds."
txt_user_data= " My name is Ivan, I have 28 years, I have a stature of 150 centimeters and I weight like 80 pounds"
# Extract entities from the text
entities = extract_entities(txt_user_data)

# Initialize an empty dictionary to store user details
user_details = {
    "name": None,
    "age": None,
    "height": None,
    "weight": None,
    "bmi":None
}

print(entities)
# Iterate through extracted entities to find user details
for entity, label in entities:
        if label == "PERSON":
            user_details["name"] = entity
        elif label == "DATE":
            if ("years" in entity or "y.o." in entity):
                user_details["age"] = int(entity.split()[0])
        elif (label == "QUANTITY"):
            if ("feet" in entity or "ft." in entity):
                user_details["height"] = feet_to_meters(float(entity.split()[0]))
            elif ("meters" in entity or "mts." in entity):
                user_details["height"] = float(entity.split()[0]) 
            elif ("centimeter" in entity or "cm." in entity):
                print("Is in centimeters",entity)
                print((float(entity.split()[0]))/100, "meters")
                user_details["height"] = (float(entity.split()[0]))/100
            elif any(unit in entity for unit in ["kg", "pounds", "kilo"]):
                if "pounds" in entity:
                    user_details["weight"] = pounds_to_kilograms(float(entity.split()[0]))
                elif any(unit in entity for unit in ["kg", "kilo"]):
                    user_details["weight"] = float(entity.split()[0])
        elif(label == "CARDINAL"):
            # If just caught a number without extra context.... 
            # Add it to all empty values? measure and guess its variable?
            for k,v in user_details.items():
                if(user_details[k]==None):
                    (float(entity.split()[0]))/100
                    x=float(entity.split()[0])
                    if (k == 'height' and x > 2):
                        user_details[k] = x/100 
                    elif (k == 'weight' and x > 160): # this is consideration of 160 and above as pounds
                        user_details[k] = pounds_to_kilograms(x)
                    else:
                        user_details[k] = x
#Process bodymass:

# verify user_details has height and weight and obtain bmi
if user_details["height"] and user_details["weight"]:
    user_details["bmi"] = calculate_bmi(user_details["weight"], user_details["height"])
    
    
print("User Details:", user_details)


[('Ivan', 'PERSON'), ('28 years', 'DATE'), ('150', 'CARDINAL'), ('80 pounds', 'QUANTITY')]
User Details: {'name': 'Ivan', 'age': 28, 'height': 1.5, 'weight': 36.28736, 'bmi': 16.127715555555554}


#  
#  
# TEST EXTRACTING SYMPTOMS!!!!!!

In [167]:
# symptoms_list
txt_user_data = " My name is John, male, I have 18 years old, and I am 1.65 meters, weight 100 kg."
txt_my_symptoms = "I have a headache, maybe a fever. Also some joint pain, and itchyness. I went to The jungle for 10 days, I eat raw food, and some insects bit me, my skin looks yellowish." #Dengue
fultext=txt_user_data+". "+txt_my_symptoms

extracted_symptoms=[]

for  s in symptoms_list:
    if (s in fultext):
        extracted_symptoms.append(s)

extracted_symptoms

['headache', 'joint pain']

#  
#  
#  
# MODELS

convert the symptoms in binary?  
Order of symptoms in symptoms affects?  
Create the list of symptoms considering the order of appearance,
therefore, when looking for them 
fill the new dataframe in that order?


The weight , is not the symptom! is just a severity!  
I required the symptoms in binaries


In [ ]:
df_sy = pd.read_csv(path+datasets[0])
df_di = pd.read_csv(path+datasets[1])
df_pr = pd.read_csv(path+datasets[2])
df_sw = pd.read_csv(path+datasets[3]) # Weights for symptoms, or labelEncoder, NOt that Relevant for NLP... so far

#Clean trailing whitespaces
df_sy = df_sy.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_sw = df_sw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# remove whitepaces
df_sy = df_sy.applymap(lambda x: x.replace(" ","_") if isinstance(x, str) else x)
df_sy = df_sy.applymap(lambda x: x.replace("__","_") if isinstance(x, str) else x)
df_sy['Disease'] = df_sy['Disease'].apply(lambda x: x.replace("_"," ") if isinstance(x, str) else x)


In [215]:
df_sw.head(1)

,Symptom,weight
0,itching,1


In [216]:
df_sy.head(1)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [223]:
# THIS IS WEIGHTS NOT IDENTIFIERS!!!!!



# Create a dictionary to map symptoms to weights
symptom_weight_mapping = df_sw.set_index('Symptom')['weight'].to_dict()

# Replace string values in df_sy with int values from the dictionary
for column in df_sy.columns[1:]:  # Skip the 'Disease' column
    df_sy[column] = df_sy[column].replace(symptom_weight_mapping)

df_sy=df_sy.fillna(0)
df_sy = df_sy.replace("foul_smell_of_urine" , 5)
df_sy = df_sy.replace("spotting__urination" , 6)
df_sy = df_sy.replace("dischromic__patches" , 6)
columns_to_convert = df_sy.columns.difference(['Disease'])
df_sy[columns_to_convert] = df_sy[columns_to_convert].astype(int)

df_sy.head(5)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,1,3,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,3,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,1,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,1,3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [226]:
# import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

import warnings
warnings.filterwarnings("ignore")

X = df_sy.drop("Disease", axis=1)
y = df_sy["Disease"]


n_fold = 7

# let's define the parameters of cross-validation (stratified 7-stock with mixing)
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state= 11)

In [227]:
clf  = RandomForestClassifier(random_state=11)

scores: np.array = cross_val_score(clf, X, y, cv=skf)
print ('all scores ', scores)
print ('mean scores ', round(scores.mean(), 2))

all scores  [0.99288762 0.99146515 0.99288762 0.9943101  0.98719772 0.99146515
 0.99002849]
mean scores  0.99


In [233]:
X = df_sy.drop('Disease', axis=1)
y = df_sy['Disease']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize and train the RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)

# Make predictions on the test set
result = rfc.predict(x_test)
result

In [234]:
result

array(['Acne', 'Acne', 'Hyperthyroidism', ..., 'Hypothyroidism',
       'Psoriasis', 'Pneumonia'], dtype=object)

In [242]:
# Create the prediction DataFrame
predict_data = {'Symptom_' + str(i + 1): symptom_weight_mapping.get(symp.replace(" ", "_"), 0)
                for i, symp in enumerate(extracted_symptoms)}
for i in range(len(predict_data),17):
    predict_data['Symptom_' + str(i + 1)]=0
    
predict_df = pd.DataFrame(predict_data, index=[0])

print(extracted_symptoms,":\n",predict_df)
# Make predictions
x = rfc.predict(predict_df)

print(x)


['headache', 'joint pain'] :
    Symptom_1  Symptom_2  Symptom_3  Symptom_4  Symptom_5  Symptom_6  \
0          3          3          0          0          0          0   

   Symptom_7  Symptom_8  Symptom_9  Symptom_10  Symptom_11  Symptom_12  \
0          0          0          0           0           0           0   

   Symptom_13  Symptom_14  Symptom_15  Symptom_16  Symptom_17  
0           0           0           0           0           0  
['Paralysis (brain hemorrhage)']


In [228]:
symptom_weight_mapping

{'itching': 1,
 'skin_rash': 3,
 'nodal_skin_eruptions': 4,
 'continuous_sneezing': 4,
 'shivering': 5,
 'chills': 3,
 'joint_pain': 3,
 'stomach_pain': 5,
 'acidity': 3,
 'ulcers_on_tongue': 4,
 'muscle_wasting': 3,
 'vomiting': 5,
 'burning_micturition': 6,
 'spotting_urination': 6,
 'fatigue': 4,
 'weight_gain': 3,
 'anxiety': 4,
 'cold_hands_and_feets': 5,
 'mood_swings': 3,
 'weight_loss': 3,
 'restlessness': 5,
 'lethargy': 2,
 'patches_in_throat': 6,
 'irregular_sugar_level': 5,
 'cough': 4,
 'high_fever': 7,
 'sunken_eyes': 3,
 'breathlessness': 4,
 'sweating': 3,
 'dehydration': 4,
 'indigestion': 5,
 'headache': 3,
 'yellowish_skin': 3,
 'dark_urine': 4,
 'nausea': 5,
 'loss_of_appetite': 4,
 'pain_behind_the_eyes': 4,
 'back_pain': 3,
 'constipation': 4,
 'abdominal_pain': 4,
 'diarrhoea': 6,
 'mild_fever': 5,
 'yellow_urine': 4,
 'yellowing_of_eyes': 4,
 'acute_liver_failure': 6,
 'fluid_overload': 4,
 'swelling_of_stomach': 7,
 'swelled_lymph_nodes': 6,
 'malaise': 6,
 'bl

In [246]:
df_sy[df_sy['Disease']=="Paralysis (brain hemorrhage)"].head(5)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
130,Paralysis (brain hemorrhage),5,3,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0
131,Paralysis (brain hemorrhage),5,3,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0
132,Paralysis (brain hemorrhage),3,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
133,Paralysis (brain hemorrhage),5,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
134,Paralysis (brain hemorrhage),5,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#  
#  
#  
#  
#  OneHotEncoding


b

In [14]:
df_sy = pd.read_csv(path+datasets[0])
df_di = pd.read_csv(path+datasets[1])
df_pr = pd.read_csv(path+datasets[2])
df_sw = pd.read_csv(path+datasets[3]) # Weights for symptoms, or labelEncoder, NOt that Relevant for NLP... so far

#Clean trailing whitespaces
df_sy = df_sy.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_sw = df_sw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# remove whitepaces
# df_sy = df_sy.applymap(lambda x: x.replace(" ","_") if isinstance(x, str) else x)
# df_sy = df_sy.applymap(lambda x: x.replace("__","_") if isinstance(x, str) else x)
# df_sy['Disease'] = df_sy['Disease'].apply(lambda x: x.replace("_"," ") if isinstance(x, str) else x)

df_sy

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df = df_sy

# Melt the dataframe so that all symptoms are in one column
df_melted = df.melt(id_vars=['Disease'], value_vars=df.columns[1:])

# Drop NA values
df_melted.dropna(subset=['value'], inplace=True)
df_melted

,Disease,variable,value
0,Fungal infection,Symptom_1,itching
1,Fungal infection,Symptom_1,skin_rash
2,Fungal infection,Symptom_1,itching
3,Fungal infection,Symptom_1,itching
4,Fungal infection,Symptom_1,itching
...,...,...,...
83461,Common Cold,Symptom_17,muscle_pain
83502,Common Cold,Symptom_17,muscle_pain
83543,Common Cold,Symptom_17,muscle_pain
83584,Common Cold,Symptom_17,muscle_pain


In [18]:

# Pivot the dataframe to create the one-hot encoded format
df_pivot = df_melted.pivot_table(index='Disease', columns='value', aggfunc=np.size, fill_value=0)

# Convert numbers to True and False
df_pivot = df_pivot.astype(bool)

# Reset column names and index
df_pivot.columns = df_pivot.columns.droplevel(0)
df_pivot.reset_index(inplace=True)
df_pivot

value,Disease,abdominal_pain,abnormal_menstruation,acidity,acute_liver_failure,altered_sensorium,anxiety,back_pain,belly_pain,blackheads,...,vomiting,watering_from_eyes,weakness_in_limbs,weakness_of_one_body_side,weight_gain,weight_loss,yellow_crust_ooze,yellow_urine,yellowing_of_eyes,yellowish_skin
0,(vertigo) Paroymsal Positional Vertigo,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,AIDS,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Acne,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,Alcoholic hepatitis,True,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
4,Allergy,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
5,Arthritis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,Bronchial Asthma,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,Cervical spondylosis,False,False,False,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,False
8,Chicken pox,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,Chronic cholestasis,True,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True


# METODO POR SYMPTOMS DIRECTO <<<<<<<<<<<<<<

In [19]:

df = df_sy

# Convert to Disease and Symptoms format
df['Symptoms'] = df.drop(columns='Disease').apply(lambda x: x.dropna().tolist(), axis=1)

# Drop the original Symptom columns
df = df[['Disease', 'Symptoms']]
df


# threshold_found_actors = math.floor(len(found_actors) * 0.75)
#         match_df = df_imdb[df_imdb['stars_name'].apply(lambda x: sum(actor in x for actor in found_actors) >= threshold_found_actors)]
        



,Disease,Symptoms
0,Fungal infection,"[itching, skin_rash, nodal_skin_eruptions, dis..."
1,Fungal infection,"[skin_rash, nodal_skin_eruptions, dischromic _..."
2,Fungal infection,"[itching, nodal_skin_eruptions, dischromic _pa..."
3,Fungal infection,"[itching, skin_rash, dischromic _patches]"
4,Fungal infection,"[itching, skin_rash, nodal_skin_eruptions]"
...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,"[vomiting, headache, nausea, spinning_movement..."
4916,Acne,"[skin_rash, pus_filled_pimples, blackheads, sc..."
4917,Urinary tract infection,"[burning_micturition, bladder_discomfort, foul..."
4918,Psoriasis,"[skin_rash, joint_pain, skin_peeling, silver_l..."


In [44]:
import math
mysymps=['headache', 'joint pain', 'bladder discomfort', 'foul smell']
mysymps=['joint pain']


threshold_found_symps = math.floor(len(mysymps) * 0.75) 
match_df = df[df['Symptoms'].apply(lambda x: sum(symps in x for symps in mysymps) >= threshold_found_symps)]
print("ORIGINAL:",len(df['Symptoms']))
print(len(match_df['Disease'].unique()))
print(match_df['Disease'].unique())
match_df.head()

ORIGINAL: 4920
41
['Fungal infection' 'Allergy' 'GERD' 'Chronic cholestasis' 'Drug Reaction'
 'Peptic ulcer diseae' 'AIDS' 'Diabetes' 'Gastroenteritis'
 'Bronchial Asthma' 'Hypertension' 'Migraine' 'Cervical spondylosis'
 'Paralysis (brain hemorrhage)' 'Jaundice' 'Malaria' 'Chicken pox'
 'Dengue' 'Typhoid' 'hepatitis A' 'Hepatitis B' 'Hepatitis C'
 'Hepatitis D' 'Hepatitis E' 'Alcoholic hepatitis' 'Tuberculosis'
 'Common Cold' 'Pneumonia' 'Dimorphic hemmorhoids(piles)' 'Heart attack'
 'Varicose veins' 'Hypothyroidism' 'Hyperthyroidism' 'Hypoglycemia'
 'Osteoarthristis' 'Arthritis' '(vertigo) Paroymsal  Positional Vertigo'
 'Acne' 'Urinary tract infection' 'Psoriasis' 'Impetigo']


,Disease,Symptoms
0,Fungal infection,"[itching, skin_rash, nodal_skin_eruptions, dis..."
1,Fungal infection,"[skin_rash, nodal_skin_eruptions, dischromic _..."
2,Fungal infection,"[itching, nodal_skin_eruptions, dischromic _pa..."
3,Fungal infection,"[itching, skin_rash, dischromic _patches]"
4,Fungal infection,"[itching, skin_rash, nodal_skin_eruptions]"


In [82]:
df.head()




,Disease,Symptoms
0,Fungal infection,"[itching, skin_rash, nodal_skin_eruptions, dis..."
1,Fungal infection,"[skin_rash, nodal_skin_eruptions, dischromic _..."
2,Fungal infection,"[itching, nodal_skin_eruptions, dischromic _pa..."
3,Fungal infection,"[itching, skin_rash, dischromic _patches]"
4,Fungal infection,"[itching, skin_rash, nodal_skin_eruptions]"


In [26]:
symptoms_list

['stomach pain',
 'mucoid sputum',
 'history of alcohol consumption',
 'knee pain',
 'obesity',
 'blood in sputum',
 'stiff neck',
 'weight loss',
 'cramps',
 'belly pain',
 'visual disturbances',
 'fast heart rate',
 'yellowing of eyes',
 'blister',
 'bloody stool',
 'indigestion',
 'malaise',
 'polyuria',
 'joint pain',
 'excessive hunger',
 'increased appetite',
 'internal itching',
 'constipation',
 'continuous feel of urine',
 'pain in anal region',
 'diarrhoea',
 'brittle nails',
 'blackheads',
 'loss of appetite',
 'irregular sugar level',
 'small dents in nails',
 'swollen blood vessels',
 'enlarged thyroid',
 'neck pain',
 'headache',
 'throat irritation',
 'dischromic  patches',
 'abnormal menstruation',
 'depression',
 'skin rash',
 'swelling joints',
 'bladder discomfort',
 'weakness of one body side',
 'sweating',
 'weight gain',
 'irritation in anus',
 'burning micturition',
 'nausea',
 'high fever',
 'rusty sputum',
 'mild fever',
 'pain during bowel movements',
 'red sp

In [30]:
df['Disease'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

#  
#  
#  


# Similarity

In [46]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


stopword = stopwords.words('english')
nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load("en_core_web_lg")

def clean_text(text):
    # Remove special characters, extra spaces, and formatting
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens


def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens


def pos_tagging(tokens):
    tagged_tokens = nltk.pos_tag(tokens)
    return tagged_tokens


def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities


def dependency_parsing(text):
    doc = nlp(text)
    dependencies = [(token.text, token.dep_, token.head.text) for token in doc]
    return dependencies



def extract_keywords(texts):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Use cosine_similarities matrix to find important keywords
    # For example, words with high TF-IDF scores or TextRank scores


/Users/ivan/opt/anaconda3/envs/sem2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-13 18:42:32.298961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [50]:
def preprocessing_text(raw_text,returned="keywords"):
    cleaned_text = clean_text(raw_text)
    print(cleaned_text)
    # Tokenization: Split the cleaned text into individual words or tokens.
    tokens = tokenize_text(cleaned_text)
    print("\n tokens:",tokens)

    # Stopword Removal: Remove common stopwords that don't provide meaningful information.
    filtered_tokens = remove_stopwords(tokens)
    print("\n filtered tokens:",filtered_tokens)
    # Part-of-Speech Tagging: Identify the parts of speech of each token.
    tagged_tokens = pos_tagging(filtered_tokens)
    print("\n tagged tokens:",tagged_tokens)
    # Named Entity Recognition (NER): Extract medical entities like symptoms and descriptions.
    entities = extract_entities(cleaned_text)
    print("\n >entities:",entities)
    # Dependency Parsing (Optional): Analyze grammatical relationships between words.
    dependencies = dependency_parsing(cleaned_text)
    print("\n >dependencies:",dependencies)
    # Keyword Extraction (Optional): Identify important keywords using TF-IDF or TextRank.
    keywords = extract_keywords(tokens)
    print("\n >keywords:",keywords)
    textes={"tokens":tokens,
           "filtered_tokens":filtered_tokens,
           "tagged_tokens":tagged_tokens,
           "entities":entities,
           "dependencies":dependencies,
           "keywords":keywords,
           }
    return textes[returned]

In [52]:
def preprocessing_text_df(raw_text):
    cleaned_text = clean_text(raw_text)
    tokens = tokenize_text(cleaned_text)
    filtered_tokens = remove_stopwords(tokens)
    filtered_tokens_str=" ".join(filtered_tokens)
    return filtered_tokens_str
    
df_di['Description_clean']=df_di['Description'].apply(preprocessing_text_df)
df_di.head()



,Disease,Description,Description_clean
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...,adverse drug reaction ADR injury caused taking...
1,Malaria,An infectious disease caused by protozoan para...,infectious disease caused protozoan parasites ...
2,Allergy,An allergy is an immune system response to a f...,allergy immune system response foreign substan...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi...",Hypothyroidism also called underactive thyroid...
4,Psoriasis,Psoriasis is a common skin disorder that forms...,Psoriasis common skin disorder forms thick red...


In [60]:
df_di['Description'].iloc[10]

" Hypoglycemia is a condition in which your blood sugar (glucose) level is lower than normal. Glucose is your body's main energy source. Hypoglycemia is often related to diabetes treatment. But other drugs and a variety of conditions — many rare — can cause low blood sugar in people who don't have diabetes."

# FIND SIMILARITY SPACY

In [78]:
dataset_raw = df_di['Description'].tolist()
dataset_clean_text = [preprocessing_text_df(x) for x in dataset_raw]
dataset_docs = [nlp(record) for record in df_di['Description_clean'].tolist()]

def find_similar_movie(summary_raw): 
    summary=preprocessing_text_df(summary_raw)
    summary_doc = nlp(summary)
    similarity_scores = [summary_doc.similarity(doc) for doc in dataset_docs]
    most_similar_index = similarity_scores.index(max(similarity_scores))
  
    df_match = df.iloc[most_similar_index]#[df['']]
    # THe best this, is to find the movie in the dataset, return title, year and link
    return df_match #most_similar_record.text

In [62]:
txt_my_symptoms = "I have a headache, maybe a fever. Also some joint pain, and itchyness. I went to The jungle for 10 days, I eat raw food, and some insects bit me, my skin looks yellowish." #Dengue
txt_my_symptoms="Hypoglycemia is a condition in which your blood sugar (glucose) level is lower than normal. Glucose is your body's main energy source. Hypoglycemia is often related to diabetes treatment"
# filtered_tokens= preprocessing_text(txt_my_symptoms,'filtered_tokens')
filtered_tokens_text= preprocessing_text_df(txt_my_symptoms)
print(filtered_tokens_text)
find_similar_movie(filtered_tokens_text)

/var/folders/n5/2wsllrvs6h9862hcd58z1ybh0000gn/T/ipykernel_4774/3429627690.py:10: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_scores = [summary_doc.similarity(doc) for doc in dataset_docs]


Disease                                               Allergy
Symptoms    [continuous_sneezing, shivering, chills, water...
Name: 10, dtype: object

#  
#  
#  
#  
# Chatgpt versino

In [63]:
import re
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("en_core_web_lg")

def preprocessing_text_df_lemma(raw_text):
    cleaned_text = clean_text(raw_text)
    tokens = tokenize_text(cleaned_text)
    filtered_tokens = remove_stopwords(tokens)
    lemmatized_tokens = lemmatize_tokens(filtered_tokens)
    filtered_tokens_str = " ".join(lemmatized_tokens)
    return filtered_tokens_str

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip().lower()
    return text

def tokenize_text(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token.lower() not in stop_words]

def lemmatize_tokens(tokens):
    return [token.lemma_ for token in nlp(" ".join(tokens))]

dataset_raw = df_di['Description'].tolist()
dataset_clean_text = [preprocessing_text_df_lemma(x) for x in dataset_raw]
df_di['Description_clean'] = dataset_clean_text

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_di['Description_clean'])

def find_similar_text(input_text): 
    input_clean = preprocessing_text_df_lemma(input_text)
    input_vector = vectorizer.transform([input_clean])
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix)
    most_similar_index = similarity_scores.argmax()
    return df_di.iloc[most_similar_index]




Disease                                                   Hypoglycemia
Description           Hypoglycemia is a condition in which your blo...
Description_clean    hypoglycemia condition blood sugar ( glucose )...
Name: 10, dtype: object


In [65]:


t1 = "I have a headache, maybe a fever. Also some joint pain, and itchyness. I went to The jungle for 10 days, I eat raw food, and some insects bit me, my skin looks yellowish." #Dengue
t2="Hypoglycemia is a condition in which your blood sugar (glucose) level is lower than normal. Glucose is your body's main energy source. Hypoglycemia is often related to diabetes treatment"
t = "My name is John, male, I have 18 years old, and I am 1.65 meters, weight 100 kg."

# filtered_tokens= preprocessing_text(txt_my_symptoms,'filtered_tokens')
_ct= find_similar_text(t)
print(_ct)

Disease                                   Dimorphic hemorrhoids(piles)
Description          Hemorrhoids, also spelled haemorrhoids, are va...
Description_clean    hemorrhoid , also spell haemorrhoid , vascular...
Name: 16, dtype: object


In [77]:
testes={
    "Fungal infection" : "My skin has been itchy and flaky with a reddish rash that doesn't seem to heal.",
"Allergy" : "Whenever I'm around pollen, my eyes water, my nose runs, and I can't stop sneezing.",
"GERD (Gastroesophageal reflux disease)" : "I often experience a burning sensation in my chest, especially after eating, and sometimes food seems to come back up my throat.",
"Chronic cholestasis" : "I've been feeling persistent itching, and my eyes and skin appear more yellow than usual.",
"Drug Reaction" : "After taking that medication, I developed a rash, swelling, and felt dizzy.",
"Peptic ulcer disease" : "There's a gnawing or burning pain in my stomach, especially on an empty stomach or at night.",
"AIDS (Acquired Immune Deficiency Syndrome)" : "I've lost weight without trying, feel constantly fatigued, and keep getting infections that I never used to have.",
"Diabetes" : "I've been extremely thirsty, urinating often, and feeling more tired than usual.",
"Gastroenteritis" : "I can't keep anything down, have diarrhea, and feel abdominal cramps.",
"Bronchial Asthma" : "I often wheeze and have shortness of breath, especially during the night or after exercise.",
"Hypertension (High blood pressure)" : "The doctor said my blood pressure readings are consistently high, even though I don't feel any specific symptoms.",
"Migraine" : "There's a throbbing pain on one side of my head, and I become sensitive to light and sound.",
"Cervical spondylosis" : "My neck feels stiff and painful, and sometimes I feel tingling or numbness in my hands.",
"Paralysis (brain hemorrhage)" : "Half of my body suddenly became weak, and I couldn't move my right arm or leg.",
"Jaundice" : "My skin and eyes have taken on a yellow tint, and my urine is much darker than usual.",
"Malaria" : "I've had recurring bouts of high fever, chills, and sweats for the past few days.",
"Chicken pox" : "I've broken out in itchy blisters all over my body, and I had a fever before the rash appeared.",
"Dengue" : "I'm feeling sudden high fever, severe headaches, and pain behind my eyes, along with joint and muscle pain.",
"Typhoid" : "I've been suffering from a persistent fever, stomach pain, headache, and I noticed a rash on my abdomen.",
"Hepatitis A" : "I've lost my appetite, feel nauseous, and my liver area hurts; plus, I've noticed a yellowing of my eyes.",
"Hepatitis B" : "I've been feeling very tired, my urine is dark, and there's a yellowish tinge to my eyes and skin.",
"Hepatitis C" : "I'm feeling a general sense of fatigue, my stomach is upset, and the doctor said my liver enzymes are elevated.",
"Hepatitis D" : "I've had jaundice, fatigue, and joint pain recently, even though I already had hepatitis B.",
"Hepatitis E" : "I've noticed jaundice, feel fatigued, and have a reduced appetite along with mild fever.",
"Alcoholic hepatitis" : "After years of heavy drinking, I've been feeling nauseous, have abdominal pain, and my eyes and skin are turning yellow.",
"Tuberculosis" : "I've had a persistent cough for weeks, sometimes coughing up blood, accompanied by weight loss, night sweats, and fatigue.",
"Common Cold" : "I've got a runny nose, sneezing, a mild sore throat, and a cough that just started.",
"Pneumonia" : "I've been feeling short of breath, with a high fever and a cough that brings up thick, colored phlegm.",
"Dimorphic hemorrhoids (piles)" : "I've noticed painful swollen veins in my rectal area, and sometimes there's blood when I wipe.",
"Heart attack" : "I suddenly felt a crushing pain in my chest that radiated to my arm and jaw, accompanied by shortness of breath.",
"Varicose veins" : "The veins in my legs have become bulgy, bluish, and often cause a dull ache.",
"Hypothyroidism" : "I've been feeling constantly tired, gaining weight, and my skin has turned dry and cold.",
"Hyperthyroidism" : "I've lost weight without trying, my heart rate has increased, and I feel jittery and hot all the time.",
"Hypoglycemia" : "I suddenly felt shaky, sweaty, and had a pounding heartbeat, and needed to eat something sweet immediately.",
"Osteoarthritis" : "The joints in my hands and knees have become painful and stiff, especially when I wake up.",
"Arthritis" : "My joints are swollen, red, and warm to the touch, and they ache constantly.",
"(Vertigo) Paroxysmal Positional Vertigo" : "Whenever I change the position of my head, I feel like the room is spinning around me.",
"Acne" : "My face, back, and chest have red, pus-filled pimples and blackheads.",
"Urinary tract infection (UTI)" : "I've had a burning sensation when I urinate, and I feel the urge to go more often than usual.",
"Psoriasis" : "There are red, itchy patches covered with silvery scales on my elbows and knees.",
"Impetigo" : "I've developed honey-colored crusted sores on my face, especially around my nose and mouth."
    
}

count=0
for k,v_text in testes.items():
    _ct= find_similar_text(v_text) #HERE YOU CALL  YOUR FUNCTION TO PROCESS v_text
    r= "✅" if _ct['Disease'].strip() == k.strip() else "❌"
    if("✅"==r):
        count+=1
    print(r , end=" ")
    print(f"Predicted: {_ct['Disease']}, Original: {k},  \n\tSentence: {v_text}")
print(f"Total {count}/{len(testes)}")


❌ Predicted: Chicken pox, Original: Fungal infection,  
	Sentence: My skin has been itchy and flaky with a reddish rash that doesn't seem to heal.
❌ Predicted: Hepatitis E, Original: Allergy,  
	Sentence: Whenever I'm around pollen, my eyes water, my nose runs, and I can't stop sneezing.
❌ Predicted: Diabetes, Original: GERD (Gastroesophageal reflux disease),  
	Sentence: I often experience a burning sensation in my chest, especially after eating, and sometimes food seems to come back up my throat.
❌ Predicted: Jaundice, Original: Chronic cholestasis,  
	Sentence: I've been feeling persistent itching, and my eyes and skin appear more yellow than usual.
✅ Predicted: Drug Reaction, Original: Drug Reaction,  
	Sentence: After taking that medication, I developed a rash, swelling, and felt dizzy.
❌ Predicted: Peptic ulcer diseae, Original: Peptic ulcer disease,  
	Sentence: There's a gnawing or burning pain in my stomach, especially on an empty stomach or at night.
❌ Predicted: Diabetes, Or

In [70]:
df['Disease'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

In [79]:

# METODO Spacy.similarity

count=0
for k,v in testes.items():
    
    filtered_tokens_text= preprocessing_text_df()
    _ct=find_similar_movie(filtered_tokens_text)

    r= "✅" if _ct['Disease'].strip() == k.strip() else "❌"
    if("✅"==r):
        count+=1
    print(r , end=" ")
    print(f"Predicted: {_ct['Disease']}, Original: {k},  \n\tSentence: {v}")
print(f"Total {count}/{len(testes)}")

✅ Predicted: Fungal infection, Original: Fungal infection,  
	Sentence: My skin has been itchy and flaky with a reddish rash that doesn't seem to heal.
❌ Predicted: Fungal infection, Original: Allergy,  
	Sentence: Whenever I'm around pollen, my eyes water, my nose runs, and I can't stop sneezing.
❌ Predicted: Fungal infection, Original: GERD (Gastroesophageal reflux disease),  
	Sentence: I often experience a burning sensation in my chest, especially after eating, and sometimes food seems to come back up my throat.
❌ Predicted: GERD, Original: Chronic cholestasis,  
	Sentence: I've been feeling persistent itching, and my eyes and skin appear more yellow than usual.
❌ Predicted: GERD, Original: Drug Reaction,  
	Sentence: After taking that medication, I developed a rash, swelling, and felt dizzy.
❌ Predicted: GERD, Original: Peptic ulcer disease,  
	Sentence: There's a gnawing or burning pain in my stomach, especially on an empty stomach or at night.
❌ Predicted: Allergy, Original: AID

In [92]:
symptoms_list

['stomach pain',
 'mucoid sputum',
 'history of alcohol consumption',
 'knee pain',
 'obesity',
 'blood in sputum',
 'stiff neck',
 'weight loss',
 'cramps',
 'belly pain',
 'visual disturbances',
 'fast heart rate',
 'yellowing of eyes',
 'blister',
 'bloody stool',
 'indigestion',
 'malaise',
 'polyuria',
 'joint pain',
 'excessive hunger',
 'increased appetite',
 'internal itching',
 'constipation',
 'continuous feel of urine',
 'pain in anal region',
 'diarrhoea',
 'brittle nails',
 'blackheads',
 'loss of appetite',
 'irregular sugar level',
 'small dents in nails',
 'swollen blood vessels',
 'enlarged thyroid',
 'neck pain',
 'headache',
 'throat irritation',
 'dischromic  patches',
 'abnormal menstruation',
 'depression',
 'skin rash',
 'swelling joints',
 'bladder discomfort',
 'weakness of one body side',
 'sweating',
 'weight gain',
 'irritation in anus',
 'burning micturition',
 'nausea',
 'high fever',
 'rusty sputum',
 'mild fever',
 'pain during bowel movements',
 'red sp

In [95]:


print(symptoms_list)

# Now in function
def find_by_symptoms(_text):
    # df used here is the one that is processed to list symptoms in one column
    print("==============")
    _extracted_symptoms=[]
    for  s in symptoms_list:
        if (s in _text.lower()):
            _extracted_symptoms.append(s)
    print("_extracted_symptoms:",_extracted_symptoms)
    threshold_found_symps = math.floor(len(_extracted_symptoms) * 0.99)#75) 
    match_df = df[df['Symptoms'].apply(lambda x: sum(symps in x for symps in _extracted_symptoms) >= threshold_found_symps)]
    return match_df





count=0
for k,v in testes.items():
    _df = find_by_symptoms(v)
    if(len(_df)>1):
        _ct= _df.iloc[0]
        r= "✅" if _ct['Disease'].strip() == k.strip() else "❌"
        if("✅"==r):
            count+=1
        print(r , end=" ")
        print(f"Predicted: {_ct['Disease']}, Original: {k},  \n\tSentence: {v}")
    else:
        print("Nothing was found, please let us know more details of your discomfort")
print(f"Total {count}/{len(testes)}")



['stomach pain', 'mucoid sputum', 'history of alcohol consumption', 'knee pain', 'obesity', 'blood in sputum', 'stiff neck', 'weight loss', 'cramps', 'belly pain', 'visual disturbances', 'fast heart rate', 'yellowing of eyes', 'blister', 'bloody stool', 'indigestion', 'malaise', 'polyuria', 'joint pain', 'excessive hunger', 'increased appetite', 'internal itching', 'constipation', 'continuous feel of urine', 'pain in anal region', 'diarrhoea', 'brittle nails', 'blackheads', 'loss of appetite', 'irregular sugar level', 'small dents in nails', 'swollen blood vessels', 'enlarged thyroid', 'neck pain', 'headache', 'throat irritation', 'dischromic  patches', 'abnormal menstruation', 'depression', 'skin rash', 'swelling joints', 'bladder discomfort', 'weakness of one body side', 'sweating', 'weight gain', 'irritation in anus', 'burning micturition', 'nausea', 'high fever', 'rusty sputum', 'mild fever', 'pain during bowel movements', 'red spots over body', 'mood swings', 'altered sensorium', 